In [36]:
# example of a dcgan on cifar10
import numpy as np
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy.random import randn
from numpy.random import randint
from keras.datasets.cifar10 import load_data
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Dropout
from matplotlib import pyplot
import keras
import seaborn as sn
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels

In [37]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import tensorflow
import os
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import normalize

In [38]:
def plot_cm(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots(constrained_layout=True)
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    return ax

In [39]:
# define the standalone generator model
def define_generator(latent_dim):
	model = Sequential()
	# foundation for 4x4 image
	n_nodes = 256 * 4 * 4
	model.add(Dense(n_nodes, input_dim=latent_dim))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Reshape((4, 4, 256)))
	# upsample to 8x8
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# upsample to 16x16
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# upsample to 32x32
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# output layer
	model.add(Conv2D(3, (3,3), activation='tanh', padding='same'))
	return model

In [40]:
def get_fake_objects(batch_size, G_input_n):
        z_noise = draw_G_random_input_noise(batch_size, G_input_n)
        G_of_z_fake_objects = final_generator.predict(z_noise)
        return z_noise, G_of_z_fake_objects

def draw_G_random_input_noise(num_samples, G_input_n):
        random_noise = np.random.uniform(-1.0, 1.0, size=[num_samples, G_input_n])
        # print("random_noise=", random_noise)
        return random_noise

In [41]:
from matplotlib import pyplot as plt

def plot_sub_objects(sample_images, file_name, full_path):
    """
    Plots image containing subplots 
    """
    try:
        figure_width = 10
        figure_height = 10
        plt.figure(figsize=(figure_width, figure_height))

        sample_size = sample_images.shape[0]
        num_items_subplot_x = 4
        num_items_subplot_y = 4

        for i in range(sample_size):
            subplot_index = i+1
            plt.subplot(num_items_subplot_x, num_items_subplot_y, subplot_index)
            image = sample_images[i, :, :, :]
            plt.imshow(image, cmap='gray')
            plt.axis('off')
        plt.savefig(full_path + '/' + file_name)
        plt.close('all')
        plt.tight_layout()
        plt.show()
    except Exception as e:
        print(e)    
        
def plot_one_object(image, file_name, full_path):
    """
    Plots image containing subplots 
    """
    try:
        # plot
        figure_width = 10
        figure_height = 10    
        plt.figure(figsize=(figure_width, figure_height))
        plt.imshow(image, cmap='gray')
        plt.axis('off')
        plt.savefig(full_path + '/' + file_name)
        plt.close('all')
        plt.tight_layout()
        plt.show()
    except Exception as e:
        print(e)    

In [31]:
G_input_n = 100
batch_size = 20
path_to_generator = './generator_model_3000.h5'

In [32]:
final_generator = define_generator(latent_dim = G_input_n)
final_generator.load_weights(path_to_generator)

In [33]:
#Define some parameters for training
batch_size = 256
num_classes = 10
epochs = 30
num_iterators = 50

In [34]:
#Load in the data (If you want undersampled dataset)
(_, _), (x_test, y_test) = load_data()

undersample_path = '../sampling/undersample/'
x_train = np.load(undersample_path + 'X9Prop0.1.npy')
x_train = x_train.astype('float32')
x_train /= 255
x_test = x_test.astype('float32')
x_test /= 255

y_train = np.load(undersample_path + 'Y9Prop0.1.npy')
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [35]:
#Set up the model function
def model_func():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same',
                     input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    
    # initiate RMSprop optimizer
    opt = tensorflow.keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)
    
    # Let's train the model using RMSprop
    model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
    return model

In [18]:
model_store = []
history_store = []
alpha_store = []
weight_store = [np.array(x_train.shape[0]*[1/x_train.shape[0]])] #Initialize as 1/n for all

for i in range(num_iterators):
    #Generate examples from GAN:
    _, generated_images = get_fake_objects(4500, 100)
    curr_x = np.concatenate((x_train, generated_images))
    curr_y = np.concatenate((y_train, keras.utils.to_categorical([9]*4500, num_classes)))
    adjusted_weight = np.concatenate((weight_store[i], [np.mean(weight_store[i])] * 4500))
    adjusted_weight /= np.sum(adjusted_weight) #Normalize
    
    #Create current model, train it, store it and it's history.
    curr_model = model_func()
    curr_history = curr_model.fit(curr_x, curr_y, batch_size=batch_size, epochs=epochs, sample_weight=adjusted_weight, validation_data=(x_test, y_test), shuffle=True)
    model_store.append(curr_model)
    history_store.append(curr_history)
    
    #Recalculate the weights, update weights
    prediction = curr_model.predict(x_train) #predict
    predict_matrix = (prediction == prediction.max(axis=1)[:,None]).astype(int) #Convert to 1-0 matrix
    match_matrix = y_train - predict_matrix #Difference to eventually generate match_vector
    match_vector = np.sum(np.abs(match_matrix), axis = 1)/2 #1 indicates no match
    error = np.sum(weight_store[i] * match_vector)/np.sum(weight_store[i]) #Calculate error
    alpha = np.log((1-error)/error) + np.log(num_classes-1) #Calulate alpha
    alpha_store.append(alpha) #Append
    new_weight = weight_store[i] * np.exp(alpha * match_vector) #Calculate new weights
    new_weight /= np.sum(new_weight) #Normalize
    weight_store.append(new_weight) #Append
    
    np.save('./GANBoostStore/Weights/new_weight' + str(i) + '.npy', new_weight)
    np.save('./GANBoostStore/Alpha/alpha' + str(i) + '.npy', alpha_store)
    curr_model.save_weights('./GANBoostStore/Model/model' + str(i) + '.h5')

Train on 50000 samples, validate on 10000 samples
Epoch 1/30


ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted: OOM when allocating tensor with shape[256,64,15,15] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node conv2d_3/Conv2D}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[loss_1/mul/_179]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted: OOM when allocating tensor with shape[256,64,15,15] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node conv2d_3/Conv2D}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

0 successful operations.
0 derived errors ignored.

In [ ]:
final_prediction_store = np.zeros_like(y_test)
for i in range(len(model_store)):
    curr_predict = model_store[i].predict(x_test)
    curr_predict_matrix = (curr_predict == curr_predict.max(axis=1)[:,None]).astype(int)
    final_prediction_store += curr_predict_matrix * alpha_store[i]
final_prediction_matrix = (final_prediction_store == final_prediction_store.max(axis=1)[:,None]).astype(int)
final_match_matrix = y_test - final_prediction_matrix
final_match_vector = np.sum(np.abs(final_match_matrix), axis = 1)/2
final_error = (len(final_match_vector)-np.sum(final_match_vector))/len(final_match_vector)

In [ ]:
#Plot confusion matrix
y_test_argmax = np.argmax(y_test, axis=1)
y_pred_argmax = np.argmax(final_prediction_matrix, axis=1)

plt.style.use('default')

plt.figure(figsize = (13,13))
plot_cm(y_test_argmax, y_pred_argmax, classes=np.array([0,1,2,3,4,5,6,7,8,9]), normalize=True,
                      title='Normalized confusion matrix')